### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [206]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# File to Load (Remember to Change These)
school_data_to_load = ("pandas_challenge/PyCitySchools/Resources/schools_complete.csv")
student_data_to_load = "pandas_challenge/PyCitySchools/Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(5)



,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [177]:
# preparing the data 
# columns to be used
clean_data = school_data_complete[['student_name', 'school_name', 'type', 'size', 'gender', 'grade', 'budget', 'math_score', 'reading_score']]
# rename columns
clean_data.rename(columns={'student_name': 'Students', 'school_name': 'School', 'type':'School Type', 
                           'size': 'Size','gender': 'Gender', 'grade': 'Grade', 'budget': 'School Budget', 
                           'math_score': 'Math Score', 'reading_score': 'Reading Score'}, inplace=True)
clean_data.head(5)

,Students,School,School Type,Size,Gender,Grade,School Budget,Math Score,Reading Score
0,Paul Bradley,Huang High School,District,2917,M,9th,1910635,79,66
1,Victor Smith,Huang High School,District,2917,M,12th,1910635,61,94
2,Kevin Rodriguez,Huang High School,District,2917,M,12th,1910635,60,90
3,Dr. Richard Scott,Huang High School,District,2917,M,12th,1910635,58,67
4,Bonnie Ray,Huang High School,District,2917,F,9th,1910635,84,97


In [230]:
# making a copy and adding  new columns 
prep_df = clean_data.copy()
prep_df[['Per Student Budget', '% Passing Reading', '% Passing Math']] = clean_data[['School Budget', 'Reading Score', 'Math Score']]
prep_df.head(5)

,Students,School,School Type,Size,Gender,Grade,School Budget,Math Score,Reading Score,Per Student Budget,% Passing Reading,% Passing Math
0,Paul Bradley,Huang High School,District,2917,M,9th,1910635,79,66,1910635,66,79
1,Victor Smith,Huang High School,District,2917,M,12th,1910635,61,94,1910635,94,61
2,Kevin Rodriguez,Huang High School,District,2917,M,12th,1910635,60,90,1910635,90,60
3,Dr. Richard Scott,Huang High School,District,2917,M,12th,1910635,58,67,1910635,67,58
4,Bonnie Ray,Huang High School,District,2917,F,9th,1910635,84,97,1910635,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [231]:
# calculate number of schools
schools = prep_df['School'].nunique()

# Calculate the ' total number of student'
students = len(clean_data['Students'])
total_budget = sum(clean_data['School Budget'].unique())

# calculate Average math & reading
math_average = clean_data['Math Score'].mean()
reading_average = clean_data['Reading Score'].mean()

# Calculate percentage of passing reading, Math & both
reading_percent = len(clean_data.loc[
    (clean_data['Reading Score'] >= 70)]) / len(clean_data['Reading Score']) * 100

math_percent = len(clean_data.loc[
    (clean_data['Math Score'] >= 70)]) / len(clean_data['Reading Score']) * 100

math_reading = len(clean_data.loc[
    (clean_data['Reading Score'] >= 70) & (clean_data['Math Score'] >= 70)]) / len(clean_data['Reading Score']) * 100

# dataframe display 
district_summary = pd.DataFrame({'Total Schools': [schools],
                                'Total students':[students],
                                 'Total Budget':[total_budget],
                                'Average Math Score': [math_average],
                                'Average Reading Score': [reading_average],
                                '% Passing Math': [math_percent],
                                '% Passing Reading': [reading_percent], 
                                '% Overall Passing': [math_reading]}).applymap('{:,.2f}'.format)
district_summary

,Total Schools,Total students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15.00,"39,170.00","24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [250]:
# school summary
prep_df['% Overall Passing'] = (prep_df['Reading Score'] >= 70) & (clean_data['Math Score'] >= 70)
# overall passing variable 
school_summary = prep_df.sort_values(['% Overall Passing'], ascending=False).groupby([
    'School', 'School Type']).agg({
    'Students': lambda x:len(x), 
    'School Budget': lambda x: x.mean(),
    'Per Student Budget': lambda x: x.sum() / len('Students'), 
    'Math Score': lambda x: x.mean(), 
    'Reading Score': lambda x: x.mean(), 
    '% Passing Math': lambda x: x.mean() * 100, 
    '% Passing Reading': lambda x: x.mean() * 100, 
    '% Overall Passing': lambda x: x.mean() * 100})

# more cleaning 

school_sorted.head(5)

,School,School Type,Students,School Budget,Math Score,Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Per Student Budget
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,94.133477,97.039828,91.334769,582.0
12,Thomas High School,Charter,1635,1043130,83.418349,83.848930,93.272171,97.308869,90.948012,638.0
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,93.392371,97.138965,90.599455,625.0
13,Wilson High School,Charter,2283,1319574,83.274201,83.989488,93.867718,96.539641,90.582567,578.0
9,Pena High School,Charter,962,585858,83.839917,84.044699,94.594595,95.945946,90.540541,609.0


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [141]:
# Top Performing Schools (By % Overall Passing)
top_five = school_sorted.head(5)
top_five

,School,School Type,Students,School Budget,Math Score,Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Per Student Budget
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,94.133477,97.039828,91.334769,582.0
12,Thomas High School,Charter,1635,1043130,83.418349,83.848930,93.272171,97.308869,90.948012,638.0
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,93.392371,97.138965,90.599455,625.0
13,Wilson High School,Charter,2283,1319574,83.274201,83.989488,93.867718,96.539641,90.582567,578.0
9,Pena High School,Charter,962,585858,83.839917,84.044699,94.594595,95.945946,90.540541,609.0


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [142]:
# Bottom  Performing Schools (By % Overall Passing)
bottom_performing = school_summary.sort_values('% Overall Passing', ascending=True)
bottom_performing.head(5)

,School,School Type,Students,School Budget,Math Score,Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Per Student Budget
10,Rodriguez High School,District,3999,2547363,76.842711,80.744686,66.366592,80.220055,52.988247,637.0
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,65.988471,80.739234,53.204476,639.0
7,Huang High School,District,2917,1910635,76.629414,81.182722,65.683922,81.316421,53.513884,655.0
5,Hernandez High School,District,4635,3022020,77.289752,80.934412,66.752967,80.862999,53.527508,652.0
8,Johnson High School,District,4761,3094650,77.072464,80.966394,66.057551,81.222432,53.539172,650.0


In [143]:
grade_nine = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '9th'].groupby('School').agg({
        'Math Score':'mean'}).rename(columns={'Math Score': '9th Grade'}))

grade_ten = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '10th'].groupby('School').agg({
        'Reading Score':'mean'}).rename(columns={'Math Score': '10th Grade'}))

grade_eleven = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '11th'].groupby('School').agg({
        'Reading Score':'mean'}).rename(columns={'Math Score': '11th Grade'}))
grade_twelve = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '12th'].groupby('School').agg({
        'Reading Score':'mean'}).rename(columns={'Math Score': '12th Grade'}))

math_grade = pd.merge(grade_nine, grade_ten, how='left', on=['School']) \
.merge(grade_eleven, how='left', on=['School']) \
.merge(grade_twelve, how='left', on=['School'])

math_grade

,9th Grade,Reading Score_x,Reading Score_y,Reading Score
School,,,,
Bailey High School,77.083676,80.907183,80.945643,80.912451
Cabrera High School,83.094697,84.253219,83.788382,84.287958
Figueroa High School,76.403037,81.408912,80.640339,81.384863
Ford High School,77.361345,81.262712,80.403642,80.662338
Griffin High School,82.044010,83.706897,84.288089,84.013699
Hernandez High School,77.438495,80.660147,81.396140,80.857143
Holden High School,83.787402,83.324561,83.815534,84.698795
Huang High School,77.027251,81.512386,81.417476,80.305983
Johnson High School,77.187857,80.773431,80.616027,81.227564


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [154]:
# Reading Score by Grade 
# Table to each grade 
grade_nine = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '9th'].groupby('School').agg({
        'Reading Score':'mean'}).rename(columns={'Reading Score': '9th Grade'}))

grade_ten = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '10th'].groupby('School').agg({
        'Reading Score':'mean'}).rename(columns={'Reading Score': '10th Grade'}))

grade_eleven = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '11th'].groupby('School').agg({
        'Reading Score':'mean'}).rename(columns={'Reading Score': '11th Grade'}))

grade_twelve = pd.DataFrame(
    clean_data.loc[clean_data['Grade'] == '12th'].groupby('School').agg({
        'Reading Score':'mean'}).rename(columns={'Reading Score': '12th Grade'}))

reading_grade = pd.merge(grade_nine, grade_ten, how='left', on=['School']) \
.merge(grade_eleven, how='left', on=['School']) \
.merge(grade_twelve, how='left', on=['School'])

reading_grade

,9th Grade,10th Grade,11th Grade,12th Grade
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
 * Overall Passing Rate (Average of the above two)

In [145]:
# adjust to the right columns 
stu_budget = prep_df.sort_values('Per Student Budget', ascending=False)
bins = [0, 584, 629, 644, 675]
group_labels = ['<$584', '$585-629', '$630-644', '$645-675']

In [146]:
prep_df['Per Student Budget'].min()

248087

In [147]:
school_summary['Performance/Budget'] = pd.cut(
    school_summary['Per Student Budget'], bins, labels=group_labels)

performance_df = school_summary.groupby(
    ['Performance/Budget']).agg({
    'Math Score': 'mean', 
    'Reading Score':'mean', 
    '% Passing Math': lambda x: x.mean() * 100, 
    '% Passing Reading': lambda x: x.mean() * 100, 
    '% Overall Passing': lambda x: x.mean() * 100})

performance_df

,Math Score,Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Performance/Budget,,,,,
<$584,83.455399,83.933814,9346.009573,9661.087678,9036.945874
$585-629,81.899826,83.155286,8713.353761,9271.820458,8141.859632
$630-644,78.518855,81.624473,7348.420890,8439.179285,6285.765555
$645-675,76.997210,81.027843,6616.481311,8113.395072,5352.685489


## Scores by School Size

* Perform the same operations as above, based on school size.

In [148]:
# Done
prep_df.reset_index()
bins = [0, 999, 1999, 5000]
group_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

prep_df['School Size'] = pd.cut(prep_df['Size'], bins, labels=group_labels)

size_score = prep_df.groupby(['School Size']).agg({'Math Score': 'mean', 
                                         'Reading Score':'mean', 
                                         '% Passing Math': lambda x: x.mean() * 100, 
                                         '% Passing Reading': lambda x: x.mean() * 100, 
                                         '% Overall Passing': lambda x: x.mean() * 100})
size_score.head(5)

,Math Score,Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


## Scores by School Type

* Perform the same operations as above, based on school type

In [149]:
# done 
type_score = prep_df.groupby(['School Type']).agg({'Math Score': 'mean', 
                                                   'Reading Score':'mean', 
                                                   '% Passing Math': lambda x: x.mean() * 100, 
                                                   '% Passing Reading': lambda x: x.mean() * 100, 
                                                   '% Overall Passing': lambda x: x.mean() * 100})
type_score.head(5)

,Math Score,Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
